In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
X_np = np.random.rand(100, 4).astype(np.float32)
y_np = np.random.randint(0, 2, size=(100,)).astype(np.int64)

X = torch.from_numpy(X_np)
y = torch.from_numpy(y_np)

In [3]:
class SimpleNN(nn.Module):
    def __init__(self, n_entries, n_hidden, n_outputs):
        super().__init__()
        self.hidden = nn.Linear(n_entries, n_hidden)
        self.output = nn.Linear(n_hidden, n_outputs)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        x = self.sigmoid(x).squeeze(1)
        return x
    
modelo = SimpleNN(n_entries=4, n_hidden=8, n_outputs=1)

In [4]:
criterion = nn.BCELoss()

optimizer = optim.Adam(modelo.parameters(), lr=0.001)

In [5]:
n_epochs = 10
batch_size = 10
dataset = torch.utils.data.TensorDataset(X, y)
loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(n_epochs):
    for batch_X, batch_y in loader:
        y_pred = modelo(batch_X)
        loss = criterion(y_pred, batch_y.float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {loss.item():.4f}')

Epoch 1/10, Loss: 0.6600
Epoch 2/10, Loss: 0.6355
Epoch 3/10, Loss: 0.6454
Epoch 4/10, Loss: 0.6800
Epoch 5/10, Loss: 0.6799
Epoch 6/10, Loss: 0.6289
Epoch 7/10, Loss: 0.6817
Epoch 8/10, Loss: 0.6699
Epoch 9/10, Loss: 0.6589
Epoch 10/10, Loss: 0.6913


In [7]:
with torch.no_grad():
    y_prob = modelo(X)
    y_pred = (y_prob > 0.5).int()
    accuracy = (y_pred == y).float().mean()
    print(f'Accuracy: {accuracy*100:.2f}')

Accuracy: 59.00


In [8]:
print(modelo)

SimpleNN(
  (hidden): Linear(in_features=4, out_features=8, bias=True)
  (output): Linear(in_features=8, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


In [9]:
import ipywidgets as widgets

capas_widget = widgets.IntSlider(min=1, max=5, description='Capas ocultas')

def try_model(cap):
    class RedDyn(nn.Module):
        def __init__(self, n_entradas, n_ocultas, n_capas):
            super().__init__()
            self.layers = nn.ModuleList()
            self.layers.append(nn.Linear(n_entradas, n_ocultas))
            for _ in range(n_capas-1):
                self.layers.append(nn.Linear(n_ocultas, n_ocultas))
            self.out = nn.Linear(n_ocultas, 1)
            self.relu = nn.ReLU()
            self.sig = nn.Sigmoid()
        def forward(self, x):
            for layer in self.layers:
                x = self.relu(layer(x))
            x = self.sig(self.out(x)).squeeze(1)
            return x

    model = RedDyn(4, 4, cap)  # 4 características, 4 neuronas por capa
    crit = nn.BCELoss()
    opt = optim.Adam(model.parameters(), lr=0.001)

    # Entrenamiento rápido (3 épocas)
    for _ in range(3):
        y_p = model(X)
        l = crit(y_p, y.float())
        opt.zero_grad(); l.backward(); opt.step()

    # Evaluación
    with torch.no_grad():
        y_p = model(X)
        preds = (y_p>0.5).int()
        acc = (preds==y).float().mean()
    print(f"Precisión con {cap} capas: {acc*100:.2f}%")

widgets.interactive(try_model, cap=capas_widget) 

interactive(children=(IntSlider(value=1, description='Capas ocultas', max=5, min=1), Output()), _dom_classes=(…